#### Imports/setup

In [1]:
%load_ext autoreload
%autoreload 2
# %reload_ext autoreload

In [2]:
import ee
import os
import pandas as pd
import geopandas as gpd
import sys

import requests # may not be needed
from sidecar import Sidecar

from modules.gee_initialize import initialize_ee 
initialize_ee()

from modules.file_to_ceo import (
get_ceo_url, 
whisp_stats_shapefile_and_ceo_url
)

from parameters.config_runtime import (
    geo_id_column, 
    out_csv,
    out_directory,
    out_shapefile,
    cols_ind_1_treecover,
    cols_ind_2_commodities,
    cols_ind_3_dist_before_2020,
    cols_ind_4_dist_after_2020,
)

from modules.agstack_to_gee import (
    start_agstack_session,
    get_agstack_token,
    register_fc_and_append_to_csv,
    # add_geo_ids_to_csv_from_lookup_df,
    add_geo_ids_to_csv_from_lookup_csv,
    add_empty_column_to_csv,
    remove_column_from_csv,
    geo_id_or_ids_to_feature_collection
)

from parameters.config_asr_url_info import asset_registry_base, user_registry_base

from parameters.config_asr_credentials import email, password

from modules.utils import (
    collection_properties_to_df,
    remove_geometry_from_feature_collection,
    get_centroid
)

from modules.tidy_tables import whisp_risk, add_eudr_risk_col, add_indicators, select_years_in_range, create_wildcard_column_list

def whisp_stats_as_df (roi): 
    %store roi
    %run process_whisp_stats.ipynb   
    return df_out

Earth Engine has been initialized with the specified credentials.


##### Set up asset registry

In [3]:
session = start_agstack_session(email,password,user_registry_base)

token = get_agstack_token(email, password, asset_registry_base)

#### Input: Geo IDs

In [4]:
# example_ids

GHA_ids = ['88bec54ad04804f5b1fafbc131266640a129be2840fa6797cda358d7e831b907', 
        '97408ef7bdac487e4a42e4abf20492b786310889fd4b0478603e2d0004c40bfb']

IDN_ids = ['c288d6c94efa9011c0e3452af9f7fa0941661377030e10d29c68764617f9816d', 
       '8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53859993522a6f05727a']

CIV_ids = ['0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f665ea83ac9aad3512918',
            'fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b84f5b9445855eef02f']

all_geo_ids= GHA_ids + IDN_ids + CIV_ids
#all_geo_ids= example_ids

#### Fetch plots and make a feature collection

In [17]:
roi = geo_id_or_ids_to_feature_collection(
    all_geo_ids=all_geo_ids, 
    geo_id_column=geo_id_column, 
    session=session)

### Whisp it

In [16]:
df = whisp_stats_as_df(roi)

Stored 'roi' (FeatureCollection)
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Datasets present in lookup:  True
geo id present
...stats processing started
...ee to df processing time: 4.004969835281372 seconds


### Display it

In [8]:
df

,Geo_id,Plot_area_ha,Geometry_type,Country,Centroid_lat,Centroid_lon,Unit,EUFO_2020,GLAD_Primary,TMF_undist,...,TMF_loss_before_2020,GFC_loss_before_2020,ESA_fire_before_2020,MODIS_fire_before_2020,TMF_deg_after_2020,TMF_loss_after_2020,GFC_loss_after_2020,MODIS_fire_after_2020,RADD_after_2020,system:index
0,88bec54ad04804f5b1fafbc131266640a129be2840fa67...,1.938,Polygon,GHA,-1.611942,6.159539,ha,0.362,0.000,0.000,...,0.450,1.552,0.00,0.000,0,0,0.000,0,0.000,0
1,97408ef7bdac487e4a42e4abf20492b786310889fd4b04...,16.602,Polygon,GHA,-2.157144,5.981149,ha,16.602,14.493,15.792,...,0.000,0.000,0.00,0.000,0,0,0.000,0,0.973,1
2,c288d6c94efa9011c0e3452af9f7fa0941661377030e10...,31.210,Polygon,IDN,103.956096,-3.054668,ha,0.000,0.000,0.000,...,31.210,31.020,31.21,6.401,0,0,0.000,0,0.000,2
3,8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53...,20.884,Polygon,IDN,103.977512,-3.083808,ha,5.841,0.000,6.472,...,3.081,9.043,0.00,0.000,0,0,0.000,0,0.000,3
4,0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f66...,8.279,Polygon,CIV,-4.101646,5.711935,ha,0.446,0.001,0.023,...,2.676,3.194,0.00,0.000,0,0,1.602,0,0.001,4
5,fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b...,3.613,Polygon,CIV,-7.507022,6.071468,ha,1.485,0.000,0.000,...,0.958,1.955,0.00,0.000,0,0,0.000,0,0.000,5


#### Export it to CSV file

In [9]:
#export output csv
df.to_csv(path_or_buf=out_csv,header=True)

print ("output csv: ", out_csv)


output csv:  /home/sepal-user/module_results/whisp/whisp_output_table.csv


##### Calculate EUDR risk category

In [10]:
# Replace with the name of your CSV file
csv_file =out_directory/'whisp_output_table.csv'

# import as a dataframe
df_w_stats = pd.read_csv(csv_file)


df_w_risk = whisp_risk(
    df = df_w_stats, 
    ind_1_pcent_threshold=10,
    ind_2_pcent_threshold=10,
    ind_3_pcent_threshold=0,
    ind_4_pcent_threshold=0
    )


##### Display table with risk columns

In [11]:
df_w_risk

,Unnamed: 0,Geo_id,Plot_area_ha,Geometry_type,Country,Centroid_lat,Centroid_lon,Unit,EUFO_2020,GLAD_Primary,...,TMF_loss_after_2020,GFC_loss_after_2020,MODIS_fire_after_2020,RADD_after_2020,system:index,Indicator_1_treecover,Indicator_2_commodities,Indicator_3_disturbance_before_2020,Indicator_4_disturbance_after_2020,EUDR_risk
0,0,88bec54ad04804f5b1fafbc131266640a129be2840fa67...,1.938,Polygon,GHA,-1.611942,6.159539,ha,0.362,0.000,...,0,0.000,0,0.000,0,yes,no,yes,no,low
1,1,97408ef7bdac487e4a42e4abf20492b786310889fd4b04...,16.602,Polygon,GHA,-2.157144,5.981149,ha,16.602,14.493,...,0,0.000,0,0.973,1,yes,no,yes,yes,low
2,2,c288d6c94efa9011c0e3452af9f7fa0941661377030e10...,31.210,Polygon,IDN,103.956096,-3.054668,ha,0.000,0.000,...,0,0.000,0,0.000,2,yes,yes,yes,no,low
3,3,8e2accea7ddbb84b7f6001e00bcb60f57f563c80633b53...,20.884,Polygon,IDN,103.977512,-3.083808,ha,5.841,0.000,...,0,0.000,0,0.000,3,yes,no,yes,no,low
4,4,0520cfac98fbc1bd7952b1c07a9f6983b83625722b6f66...,8.279,Polygon,CIV,-4.101646,5.711935,ha,0.446,0.001,...,0,1.602,0,0.001,4,yes,yes,yes,yes,low
5,5,fa2aff0d60cf1bc0e1f1dd4b91daf932940c31c021ca1b...,3.613,Polygon,CIV,-7.507022,6.071468,ha,1.485,0.000,...,0,0.000,0,0.000,5,yes,yes,yes,no,low


##### Export table with risk columns to csv 

In [12]:
output_file=out_directory/'whisp_output_table_w_geo_id_w_risk.csv'

# export to a csv
df_w_risk.to_csv(output_file)
